In [ ]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [1]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [7]:
## Define Model here
# Need to define it up top to prevent overwriting it by mistake
import torch

class AudGenModel(torch.nn.Module):
    def __init__(self):
        super(AudGenModel, self).__init__()

        chanMult = 56
        kernel = 4
        stride = 2
        self.relu = torch.nn.LeakyReLU(0.2, inplace=True)
        self.sigmoid = torch.nn.Sigmoid()

        self.conv1 = torch.nn.Conv2d(in_channels = 1, out_channels = chanMult, kernel_size = kernel ,stride = stride ,padding = 1)
        self.conv2 = torch.nn.Conv2d(in_channels = chanMult, out_channels = 2 * chanMult, kernel_size = kernel ,stride = stride ,padding = 1)
        self.conv3 = torch.nn.Conv2d(in_channels = 2 * chanMult, out_channels = 4 * chanMult, kernel_size = kernel ,stride = stride ,padding = 1)
        self.output = torch.nn.Linear(chanMult * 9 * 4, 1)


    def forward(self, input):


        ################################
        # Please fill in your code here:
        ################################
        
        x = self.relu(self.conv1(input))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = torch.flatten(x,1) #keep the 1st dim since it's the batch
        #x = self.relu(self.hidden(x))
        #x = torch.functional.dropout(x,0.25)
        x = self.output(x)
        out = self.sigmoid(x)

        
        return out
    
a = AudGenModel()

In [15]:
## Model Training 1 ##
import pandas as pd
import json
import copy
import csv
import pickle
import torchvision
import itertools
import torch
import glob
import random



#get number after # but before space afterwards

#generate filepaths
#filepath_mp4 = ENV_FOLDER_DATA + filesuf_mp4
#folderpath_normmp4 = ENV_FOLDER_DATA_PROC + 'norm\\' + str_epnum + '\\'
#os.makedirs(folderpath_normmp4, exist_ok=True)
#filepath_normmp4 = folderpath_normmp4 + str_epnum + '.mp4'
#folderpath_eps = ENV_FOLDER_DATA_PROC + 'eps\\' + str_epnum + '\\'
#os.makedirs(folderpath_eps, exist_ok=True)
#folderpath_clips = ENV_FOLDER_DATA_PROC + 'clips\\' + str_epnum + '\\'
#os.makedirs(folderpath_clips, exist_ok=True)
folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
os.makedirs(folderpath_clipinfo, exist_ok=True)
folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
os.makedirs(folderpath_sourcedf, exist_ok=True)
#filepath_sourcedf = folderpath_sourcedf + str_epnum + '.pickle'
#folderpath_wavsplit = folderpath_eps + 'wavsplit\\'
#os.makedirs(folderpath_wavsplit, exist_ok=True)
#filepath_segmentation = ENV_FOLDER_DATA_PROC + 'segmentation\\' + str_epnum + '.psv'
folderpath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\'
folderpath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' 
filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

## get overlap between sourcedf files and targetdf files
list_eps = []

#note - source is actually a dict of dict of tensors
dict_sourcedf = {}
list_sourcedf_eps = []
for path_pickle in glob.glob(folderpath_sourcedf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_sourcedf_eps.append(int_epnum)
    dict_sourcedf[int_epnum] = dict_temp

#note - target is actually a df
dict_targetdf = {}
list_targetdf_eps = []
for path_pickle in glob.glob(folderpath_targetdf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_targetdf_eps.append(int_epnum)
    dict_targetdf[int_epnum] = dict_temp

#populate matches
for src_ep in list_sourcedf_eps:
    if src_ep in list_targetdf_eps:
        list_eps.append(src_ep)

#40 data elements
list_datacols = []
for i in range(40):
    list_datacols.append('d' + str(i))

#define model pieces
NUM_EPOCHS = 50
TRAINING_TYPE = 'p'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform=torchvision.transforms.Normalize( mean=(255*0.5), std=(255*0.5) )
optimizer = torch.optim.Adam(a.parameters(), lr=0.0008, betas=(0.65, 0.999))
criterion = torch.nn.BCELoss()


##Epoch Loop
for e in range(NUM_EPOCHS):
    #make a copy of the episode list to pop out 2 at a time
    list_epocheps = copy.deepcopy(list_eps)
    while len(list_epocheps) > 0:
        #load two or 3 episodes into memory at a time
        list_tempeps = []
        for i in range(2):
            if len(list_epocheps) > 0:
                list_tempeps.append(list_epocheps.pop())
        #load pratt files into memory
        dict_target = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_targetdf[tempeps]['path'], 'rb') as file:
                dict_target[tempeps] =  pickle.load(file)
                    
        #load video files into memory
        dict_source = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_sourcedf[tempeps]['path'], 'rb') as file:
                dict_temp = pickle.load(file)
                dict_source[tempeps] = dict_temp

        #create a list of eps / clips and shuffle
        listtup_epseg = []
        for key,val in dict_source.items():
            for key2,val2 in val.items():
                tup_epseg = [key,key2]
                listtup_epseg.append( tup_epseg)
        random.shuffle(tup_epseg)

        #loop for data elements
        for tup_epseg in listtup_epseg:
            int_epnum = tup_epseg[0]
            int_segnum = tup_epseg[1]
            #feed values into the predict / loss / optimize cycle
            inputs = None
            labels = None
            inputs = dict_source[int_epnum][int_segnum]
            #pad inputs
            #for x in range(40 - len()):
            #    inputs.append( torch.empty((1,360,640), dtype=torch.int8) )
            #inputs = torch.stack(inputs[:40], 0)
            #transform inputs
            inputs = transform(inputs.to(torch.float))
            

            #check if labels exist
            label_df = dict_target[int_epnum].query('seg==' + str(int_segnum) + ' & type=="' + TRAINING_TYPE + '"' )
            if len(label_df) == 0:
                print('No labels for ep:' + str(int_epnum) + ' seg:' + str(int_segnum))
                continue
            labels = torch.tensor(label_df[list_datacols].fillna(0).values)

            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = a(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # print statistics
        #if i % 1000 == 0:
        #    print(f'Epoch={epoch + 1} Iter={i + 1:5d} Loss={loss.item():.3f}')
        #    running_loss = 0.0




    #define a function that, given a targetdf value, look up the source - if both exists, use the data to train the model

#end loop through data

In [12]:
dict_target[int_epnum]

,epnum,seg,type,m,d0,d1,d2,d3,d4,d5,...,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39
0,568,10,p,165.069969,NaN,301.563458,160.234094,202.516353,251.838023,NaN,...,0,0,0,0,0,0,0,0,0,0
1,568,10,i,77.730583,53.412004,83.021002,79.706681,81.123379,85.150961,80.535664,...,0,0,0,0,0,0,0,0,0,0
2,568,10,s,0.103940,NaN,0.079449,0.184175,NaN,0.054874,NaN,...,0,0,0,0,0,0,0,0,0,0
3,568,10,j,0.025482,NaN,0.018749,0.053431,0.100098,0.010418,NaN,...,0,0,0,0,0,0,0,0,0,0
4,568,10,h,7.332125,-1.737083,13.626913,7.216890,0.295520,11.956649,1.957184,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,568,995,p,165.069969,281.018923,156.685845,NaN,154.109597,163.191547,148.377749,...,NaN,NaN,NaN,200.126175,147.085506,253.048958,211.319408,151.01921,124.622928,126.902647
5811,568,995,i,77.730583,45.317132,83.272632,67.277186,84.194703,83.031170,71.952709,...,47.186942,44.809534,43.44847,82.971886,83.847621,73.182405,49.531993,78.17987,87.668509,86.578639
5812,568,995,s,0.103940,0.370050,0.146684,NaN,0.056085,0.085791,0.268858,...,NaN,NaN,NaN,0.125616,0.103498,0.271799,NaN,0.193432,0.110596,0.066256
5813,568,995,j,0.025482,0.050324,0.020268,NaN,0.010784,0.017272,0.061405,...,NaN,NaN,NaN,0.05787,0.025018,0.057206,0.100295,0.076751,0.010229,0.012673
